In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
from sklearn import metrics
from sklearn import calibration
from dcurves import dca as decision_curve_analysis
from utils import get_mean_results

plt.rcParams['savefig.dpi'] = 500
# set xticks fontsize
plt.rcParams['xtick.labelsize'] = 18
# set yticks fontsize
plt.rcParams['ytick.labelsize'] = 18
# set labels fontsize
plt.rcParams['axes.labelsize'] = 22
# set legend fontsize
plt.rcParams['legend.fontsize'] = 12
# set labels weight to bold
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['figure.figsize'] = (10, 10)

# set rcparams for grid axs[1].grid(linestyle='--', alpha=0.7, linewidth=1)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.alpha'] = 0.7
plt.rcParams['grid.linewidth'] = 1

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
NUMBER_BINS = 10

In [ ]:
model = 'LogisticRegression'
model_name = 'LR'

In [ ]:
experiments = {
    'cross-validation' : {
        '../tests/test1/cv-PT' : 'LR - All Features',
        '../tests/test1/cv-PT-reduced-13/' : 'LR - 13 Best Features',
    },
    'train-test' : {
        '../tests/test1/train-PT-test-US' : 'LR - All Features',
        '../tests/test1/train-PT-test-US-reduced-13' : 'LR - 13 Best Features',
    }
}

In [ ]:
map_names = {
    'cross-validation' : 'CV PT',
    'train-test' : 'Train PT - Test CSL'
}

In [ ]:
ticks = np.linspace(0, 1, NUMBER_BINS + 1)

## get ROCs and Calibration Curves

In [ ]:
data = {}
for data_split, exps_paths in experiments.items():
    preds_values = {}
    for experiment_dir, experiment_name in exps_paths.items():
        mean_fpr = np.linspace(0, 1, 100)
        tpr_rates = []
        roc_scores = []
        prob_true_list = []
        prob_pred_list = []
        c_slopes = []
        c_intercepts = []
        # plt.figure(figsize=(10, 10))
        for exp in os.listdir(experiment_dir):
            try:
                preds_path = os.path.join(experiment_dir, exp, 'predictions')
                models_preds = [f for f in os.listdir(preds_path) if model in f]
                for preds_file in models_preds:
                    aux_path = os.path.join(preds_path, preds_file)
                    preds_df = pd.read_csv(aux_path)
                    # roc metric
                    roc_score = metrics.roc_auc_score(preds_df.y_true, preds_df.y_proba_1)
                    roc_scores.append(roc_score)
                    # roc curve
                    fpr_proba, tpr_proba, threshold_proba = metrics.roc_curve(preds_df.y_true, preds_df.y_proba_1)
                    interp_tpr = np.interp(mean_fpr, fpr_proba, tpr_proba)
                    interp_tpr[0] = 0.0
                    tpr_rates.append(interp_tpr)

                    # calibration curve
                    prob_true, prob_pred = calibration.calibration_curve(preds_df.y_true, preds_df.y_proba_1, n_bins=NUMBER_BINS)
                    slope, intercept = np.polyfit(prob_pred, prob_true, 1)
                    c_slopes.append(slope)
                    c_intercepts.append(intercept)
                    # complete the list with nan if the length is less than NUMBER_BINS
                    if len(prob_true) < NUMBER_BINS:
                        prob_true = np.concatenate((prob_true, np.full(NUMBER_BINS - len(prob_true), np.nan)))
                        prob_pred = np.concatenate((prob_pred, np.full(NUMBER_BINS - len(prob_pred), np.nan)))
                    prob_true_list.append(prob_true)
                    prob_pred_list.append(prob_pred)
            except:
                pass

        try:
            mean_tpr = np.mean(tpr_rates, axis=0)
            mean_tpr[-1] = 1.0

            mean_prob_true = np.nanmean(prob_true_list, axis=0)
            mean_prob_pred = np.nanmean(prob_pred_list, axis=0)

            preds_values[experiment_name] = {
                'fpr' : mean_fpr,
                'tpr' : mean_tpr,
                'roc_auc_mean' : np.mean(roc_scores).round(3),
                'roc_auc_std' : np.std(roc_scores).round(3),
                'prob_true' : mean_prob_true,
                'prob_pred' : mean_prob_pred,
                'c_slopes_mean' : np.mean(c_slopes).round(3),
                'c_slopes_std' : np.std(c_slopes).round(3),
                'c_intercepts_mean' : np.mean(c_intercepts).round(3),
                'c_intercepts_std' : np.std(c_intercepts).round(3),
            }
        except:
            pass
    data[data_split] = preds_values

## plots for cross-validation

In [ ]:
plt.figure()
sns.set_style("whitegrid")
for model, result in data['cross-validation'].items():
    lw = 3
    plt.plot(
        result['fpr'], 
        result['tpr'], 
        label=f'[ROC AUC= {result["roc_auc_mean"]} $\pm$ {result["roc_auc_std"]}] {model}',
        lw=lw)
plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'grey', label='Random Classifier')
    
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.xticks(ticks)
plt.yticks(ticks)
plt.grid(linestyle='--', alpha=0.7, linewidth=1)
plt.savefig(f'../images/cross-validation-ROC-models.png', bbox_inches='tight')
plt.savefig(f'../images/cross-validation-ROC-models.svg', bbox_inches='tight')
plt.savefig(f'../images/cross-validation-ROC-models.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure()
sns.set_style("whitegrid")
for model, result in data['train-test'].items():
    lw = 3
    plt.plot(
        result['fpr'], 
        result['tpr'], 
        label=f'(ROC AUC= {result["roc_auc_mean"]} $\pm$ {result["roc_auc_std"]}) - {model}',
        lw=lw)
plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'grey', label='Random Classifier')
    
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.xticks(ticks)
plt.yticks(ticks)
plt.grid(linestyle='--', alpha=0.7, linewidth=1)
plt.savefig(f'../images/train-test-ROC-models.png', bbox_inches='tight')
plt.savefig(f'../images/train-test-ROC-models.svg', bbox_inches='tight')
plt.savefig(f'../images/train-test-ROC-models.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 7))
sns.set_style("whitegrid")
# cross validation
for model, result in data['cross-validation'].items():
    lw = 3
    axs[0].plot(
        result['fpr'], 
        result['tpr'], 
        label=f'[ROC AUC= {result["roc_auc_mean"]} $\pm$ {result["roc_auc_std"]}] {model}',
        lw=lw)
axs[0].plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'grey', label='Random Classifier')
axs[0].set_xlabel('False Positive Rate')
axs[0].set_ylabel('True Positive Rate')
axs[0].legend(loc='lower right')
axs[0].set_title('Cross-Validation PT', fontsize=24, fontweight='bold')
# train-test
for model, result in data['train-test'].items():
    lw = 3
    axs[1].plot(
        result['fpr'], 
        result['tpr'], 
        label=f'[ROC AUC= {result["roc_auc_mean"]} $\pm$ {result["roc_auc_std"]}] {model}',
        lw=lw)
axs[1].plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'grey', label='Random Classifier')
axs[1].set_xlabel('False Positive Rate')
axs[1].set_ylabel('True Positive Rate')
axs[1].legend(loc='lower right')
axs[1].set_title(map_names['train-test'], fontsize=24, fontweight='bold')
plt.savefig(f'../images/ROC-models.png', bbox_inches='tight')
plt.savefig(f'../images/ROC-models.svg', bbox_inches='tight')
plt.savefig(f'../images/ROC-models.pdf', bbox_inches='tight')
plt.show()

## calibration plots

In [ ]:
plt.figure()
sns.set_style("whitegrid")
plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color='grey', label='Perfectly Calibrated')
for model, result in data['cross-validation'].items():
    lw = 3
    plt.plot(
        result['prob_pred'], 
        result['prob_true'], 
        label=f'[C-S={result["c_slopes_mean"]}$\pm${result["c_slopes_std"]},C-I={result["c_intercepts_mean"]}$\pm${result["c_intercepts_std"]}] {model}',
        lw=lw, marker='o')
    
plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
plt.xticks(ticks)
plt.yticks(ticks)
plt.grid(linestyle='--', alpha=0.7, linewidth=1)
plt.legend(loc='lower right')
plt.savefig(f'../images/cross-validation-calibration-models.png', bbox_inches='tight')
plt.savefig(f'../images/cross-validation-calibration-models.svg', bbox_inches='tight')
plt.savefig(f'../images/cross-validation-calibration-models.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure()
sns.set_style("whitegrid")
plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color='grey', label='Perfectly Calibrated')
for model, result in data['train-test'].items():
    lw = 3
    plt.plot(
        result['prob_pred'], 
        result['prob_true'], 
        label=f'[C-S={result["c_slopes_mean"]},C-I={result["c_intercepts_mean"]}] {model}',
        lw=lw, marker='o')
    
plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
plt.xticks(ticks)
plt.yticks(ticks)
plt.grid(linestyle='--', alpha=0.7, linewidth=1)
plt.legend()
plt.savefig(f'../images/train-test-calibration-models.png', bbox_inches='tight')
plt.savefig(f'../images/train-test-calibration-models.svg', bbox_inches='tight')
plt.savefig(f'../images/train-test-calibration-models.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 7))
sns.set_style("whitegrid")
# cross validation
axs[0].plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'grey', label='Perfectly Calibrated')
for model, result in data['cross-validation'].items():
    lw = 3
    axs[0].plot(
        result['prob_pred'], 
        result['prob_true'], 
        label=f'[C-S={result["c_slopes_mean"]},C-I={result["c_intercepts_mean"]}] {model}',
        lw=lw, marker='o')
axs[0].set_xlabel('Predicted Probability')
axs[0].set_ylabel('True Probability')
axs[0].legend(loc='upper left')
axs[0].set_title('Cross-Validation PT', fontsize=24, fontweight='bold')
axs[0].grid(linestyle='--', alpha=0.7, linewidth=1)
# train-test
axs[1].plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'grey', label='Perfectly Calibrated')
for model, result in data['train-test'].items():
    lw = 3
    lw = 3
    axs[1].plot(
        result['prob_pred'], 
        result['prob_true'], 
        label=f'[C-S={result["c_slopes_mean"]},C-I={result["c_intercepts_mean"]}] {model}',
        lw=lw, marker='o')
    
axs[1].set_xlabel('Predicted Probability')
axs[1].set_ylabel('True Probability')
axs[1].legend(loc='upper left')
axs[1].set_title(map_names['train-test'], fontsize=24, fontweight='bold')
axs[1].grid(linestyle='--', alpha=0.7, linewidth=1)
plt.savefig(f'../images/calibration-curves-models.png', bbox_inches='tight')
plt.savefig(f'../images/calibration-curves-models.svg', bbox_inches='tight')
plt.savefig(f'../images/calibration-curves-models.pdf', bbox_inches='tight')
plt.show()

### Confusion Matrices

In [ ]:
def get_cm(experiment_dir, model_name):
    tps, fps, tns, fns, roc_aucs = [], [], [], [], []
    for run in sorted(os.listdir(experiment_dir)):
        run_dir = os.path.join(experiment_dir, run)
        try:
            with open(os.path.join(run_dir, 'results.json')) as f:
                results = json.load(f)
            roc_aucs += results[model_name]['roc_auc_score']
            tps += results[model_name]['tp']
            fps += results[model_name]['fp']
            tns += results[model_name]['tn']
            fns += results[model_name]['fn']
        except Exception as e:
            pass

    cm = np.array([[np.mean(tns), np.mean(fps)], [np.mean(fns), np.mean(tps)]])
    std_cm = np.array([[np.std(tns), np.std(fps)], [np.std(fns), np.std(tps)]])

    if std_cm[0][0] == 0:
        group_counts = ['{0:0.2f}'.format(value) for value in cm.flatten()]
        percentages_cm = (cm.T / cm.sum(axis=1)).T
        group_percentages = ['{0:.2%}'.format(value) for value in percentages_cm.flatten()]
    else:
        group_counts = ['{0:0.2f} ± {1:0.2f}'.format(value, std) for value, std in zip(cm.flatten(), std_cm.flatten())]
        percentages_cm = (cm.T / cm.sum(axis=1)).T
        # add percentages std
        percentages_std = (std_cm.T / cm.sum(axis=1)).T
        group_percentages = ['{0:.2%} ± {1:.2%}'.format(value, std) for value, std in zip(percentages_cm.flatten(), percentages_std.flatten())]
    #     group_percentages = ['{0:.2%}'.format(value) for value in percentages_cm.flatten()]

    labels = [f'{v1}\n({v2})' for v1, v2 in zip(group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    return cm, labels

In [ ]:
NUMBER_MODELS = 2
NUMBER_TESTS = 2

In [ ]:
fig, axs = plt.subplots(ncols=NUMBER_TESTS, nrows=NUMBER_MODELS, figsize=(15, 10))
for i, (name, exp_models) in enumerate(experiments.items()):
    for j, (exp_dir, model_name) in enumerate(exp_models.items()):
        cm, labels = get_cm(exp_dir, 'LogisticRegression')
        sns.heatmap(cm, annot=labels, fmt='', cmap='Blues', annot_kws={"size": 15, 'weight': 'bold'},  ax=axs[j, i])
        axs[j, i].set_title(f'[{map_names[name]}] {model_name}'.replace('Combination', 'Comb'), fontsize=16, fontweight='bold')
        axs[j, i].set_xlabel('Predicted Label', fontsize=14)
        axs[j, i].set_ylabel('True Label', fontsize=14)
        colorbar = axs[j, i].collections[0].colorbar
        colorbar.ax.tick_params(labelsize=12)
        # set ticks font size to 15
        axs[j, i].tick_params(axis='both', which='major', labelsize=14)
        axs[j, i].set_xticklabels(['VD', 'CS'])
        axs[j, i].set_yticklabels(['VD', 'CS'])
        # change vertical space between subplots
        plt.subplots_adjust(hspace=0.3)

plt.savefig(f'../images/CM-models.png', bbox_inches='tight')
plt.savefig(f'../images/CM-models.svg', bbox_inches='tight')
plt.savefig(f'../images/CM-models.pdf', bbox_inches='tight')

## Decision Curve analysis

In [ ]:
thresholds = np.linspace(0, 1, 101)

In [ ]:
def get_dca_values(path):
    dca_values = {
        'y_proba_1': [],
        'all' : [],
        'none' : [],
        'thresholds' : []
    }
    for exp in os.listdir(path):
        preds_path = os.path.join(path, exp, 'predictions')
        for preds in [preds for preds in os.listdir(preds_path) if 'LogisticRegression' in preds]:
            predictions = pd.read_csv(os.path.join(preds_path, preds))
            dca_df = decision_curve_analysis(
                data = predictions,
                outcome = 'y_true',
                modelnames = ['y_proba_1'],
                thresholds = thresholds
            )
            # save values
            for model in dca_df.model.unique():
                dca_values[model].append(dca_df.loc[dca_df['model'] == model, 'net_benefit'].values)
                dca_values['thresholds'].append(dca_df.loc[dca_df['model'] == model, 'threshold'].values)

    for key, value in dca_values.items():
        print(key)
        dca_values[key] = np.mean(value, axis=0)
    return dca_values

In [ ]:
values = {}
for val_type, exps in experiments.items():
    aux = {}
    for path_dir, title in exps.items():
        dca_values = get_dca_values(path_dir)
        aux[title] = dca_values
    values[val_type] = aux

In [ ]:
names = {
    'y_proba_1' : 'LR',
    'all' : 'Treat as C-Section',
    'none' : 'Treat as Vaginal Delivery'
}

In [ ]:
dca_values = values['cross-validation']

In [ ]:
dca_values['LR - All Features'].keys()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 7))
for index, (val_type, value_info) in enumerate(values.items()):
    dca_values = list(value_info.values())[0]
    cmap = plt.get_cmap('Greys')
    axs[index].plot(dca_values['thresholds'], dca_values['all'], label=f'{names["all"]}', linewidth=2, color=cmap(0.8))
    axs[index].plot(dca_values['thresholds'], dca_values['none'], label=f'{names["none"]}', linewidth=2, color=cmap(0.6))
    # setup colors
    for title, dca_values in value_info.items():
        axs[index].plot(dca_values['thresholds'], dca_values['y_proba_1'], label=f'{names["y_proba_1"]} - {title}', linewidth=3)
    axs[index].set_xlabel('Threshold')
    axs[index].set_ylabel('Net Benefit')
    axs[index].set_title(f'{map_names[val_type]}'.replace('CV', 'Cross-Validation'), fontsize=24, fontweight='bold')
    axs[index].set_xticks(np.linspace(0, 1, 11))
    axs[index].set_yticks(np.arange(-0.05, 0.35, 0.05))
    axs[index].set_ylim([-0.05, 0.35])
    axs[index].set_xlim([0, 1])
    axs[index].legend(fontsize=13)
    axs[index].grid(linestyle='--', alpha=0.7, linewidth=1)
# plt.grid(linestyle='--', linewidth=1, alpha=0.7)
plt.savefig(f'../images/DCA-models.png', bbox_inches='tight')
plt.savefig(f'../images/DCA-models.svg', bbox_inches='tight')
plt.savefig(f'../images/DCA-models.pdf', bbox_inches='tight')
plt.show()